In [13]:
import sys
sys.path.append('../')


import numpy as np
from utilities.Utils import jacob

import matplotlib.pyplot as plt
import pandas as pd
import cmath

theta = [np.random.uniform(0.,1.) for _ in range(4)]
weights = [np.random.uniform(0.,1.) for _ in range(4)]
weights /= np.sum(weights)

log_theta = np.log(theta)

log_theta[0] = log_theta[0] * -1
log_theta[3] = log_theta[3] * -1

print(log_theta)

log_log = np.log(np.abs(log_theta))

deltas = np.log(weights) + log_log

expectation_log_theta = 0
for i in range(len(theta)): 
    expectation_log_theta += weights[i] * np.log(theta[i])

print(f"Using the naive approach: {expectation_log_theta}")

print(f"ξ using the complex valued log: {-1*np.exp(jacob(deltas)[-1])}")







[ 0.32199416 -1.01074615 -0.95813875  0.10323297]
Using the naive approach: -0.5691936373111984
ξ using the complex valued log: -0.5691936373111984
